# Second-order ODE Solver

**Important concepts:**

- Understanding how to write a second-order ODE as a recursive solution using the Forward Euler formula.
- Identification of stable and unstable solutions.
- Understanding that the introduction of more non-linear terms (e.g., resistance) will tighter restrict timesteps for applying algorithms like the Forward Euler formula to solve ODEs.

In the previous chapter, we discussed how one can use the Taylor series to solve an ordinary differential equation of the first order. In this chapter, we will extend the same method to solve the second-order differential equation that governs the motion of a simple pendulum, as shown in the figure below.

![Pendulum](assets/pendulum.png)

The string has a length $L$ and the bob has a mass $m$. The restoring force (the one that brings the pendulum to the central position) is the component of the gravitational force along the tangent to the pendulum’s trajectory. This is opposed (in the presence of air) by air resistance that is proportional to the instantaneous velocity of the bob. A summation of forces and the application of Newton’s second law leads to:

$$\underbrace{\frac{d^2\theta}{dt^2}}_{\text{acceleration}} + \overbrace{\frac{k}{m}\frac{d\theta}{dt}}^{\text{resistance}} + \underbrace{\frac{g}{L}\sin\theta}_{\text{gravitational forces}} = 0$$

Where $\theta$ is the angular displacement, $g$ is the acceleration due to gravity, $t$ is the time, and $k$ is the coefficient of air resistance.

## A simpler situation

Let’s start with a simpler situation where the pendulum is in a vacuum (i.e., $k = 0$). To get the pendulum moving, one leaves it at an angular displacement $\theta$ from rest. Therefore, the equation now reads:

$$\frac{d^2\theta}{dt^2} + \frac{g}{L}\sin\theta = 0$$

With the initial conditions set, as an example, to:

$$\frac{d\theta}{dt}(0) = 0 \quad \text{and} \quad \theta(0) = \frac{\pi}{3} \quad (\text{say})$$

We can rewrite this second order differential equation into a system of two linear order differential equations by adding $\dot{\theta}$ defined as ${{d\theta } \over {dt}} = \dot{\theta}$:

$$\begin{cases}
{{d\dot{\theta} } \over {dt}} + {g \over L}\sin \theta  = 0 \\
{{d\theta } \over {dt}} = \dot{\theta} \\
\end{cases}$$

with 

$${\dot{\theta}}(0) = 0 \quad \text{and} \quad \theta(0) = \frac{\pi}{3} \quad (\text{say})$$

Discretizing these equation leads to:
$$\begin{cases}
{{d\dot{\theta}_n } \over {dt}} + {g \over L}\sin \theta_n  = 0 \\
{{d\theta_n } \over {dt}} = \dot{\theta}_n \\
\end{cases}$$

Try rewriting the above using the Forward Euler scheme. You should get:

$$\begin{cases}
{{{\dot{\theta} _{n + 1}} - {\dot{\theta} _n}} \over {\Delta t}} + {g \over L}\sin \theta_{n}  = 0 \\
{{{\theta _{n + 1}} - {\theta _n}} \over {\Delta t}} = {\dot{\theta}_n}
\end{cases}$$

One thus obtains a coupled system of equations governing the motion of the pendulum:

$$\begin{cases}
\dot{\theta}_{n+1} = \dot{\theta}_n - \Delta t\dfrac{g}{L}\sin\theta_n \\
\theta_{n+1} = \theta_n + \dot{\theta}_{n}\Delta t
\end{cases}$$

~~Given the explicit nature of the above equations, one can use $\dot{\theta}(0) = 0$ to obtain $\dot{\theta}_{n+1}$ for a time-step $-\Delta t$, followed by using $\dot{\theta}_{n+1}$ and $\theta(0)$ to obtain $\theta_{n+1}$.~~

## A little complicated

Now, let’s include air resistance:

$$\ddot{\theta} + \frac{k}{m}\dot{\theta} + \frac{g}{L}\sin\theta = 0, \text{ with } \dot{\theta}(0) = 0 \text{ and } \theta(0) = \frac{\pi}{3}$$

The formula does not change significantly:

$$\dot{\theta}_{n+1} = \dot{\theta}_n - \Delta t \frac{k}{m}\dot{\theta}_n - \Delta t \frac{g}{L}\sin\theta_n$$

$$\theta_{n+1} = \theta_n + \dot{\theta}_{n+1}\Delta t$$

However, it is important to mention that the exact (analytical) solution is more difficult to obtain.

## Plotting the results

From the above equations, it is apparent that the choice of time-step will determine the accuracy with which the trajectory of the pendulum will be resolved. Take a look at the results for time steps of $0.05$, $0.1$, $0.25$, $0.5$, and $1$. How will you decide which time step is ideal for this solver?

The plot below shows the result for $L = 1 m$, $m = 1 kg$. We can vary $k$ and $\Delta t$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

In [23]:
def plot(k,dt):
    T = 5           # total duration of the simulation
    N = int(T/dt)        # in how much sub pieces we should break a 1sec interval
    #dt = T / N      # dt
    g = 9.81        # acceleration of gravity
    L = 1           # pendulum rope length
    #k = 0.8         # air resistance coefficient
    m = 1           # mass of the pendulum

    theta = [np.pi / 3]     # initial angle
    theta_dot = [0]         # initial angular velocity
    t = [0]

    for i in range(N * T):
        theta.append(theta_dot[-1] * dt + theta[-1])
        theta_dot.append(theta_dot[-1] - theta_dot[-1] * dt * k / m - np.sin(theta[-1]) * dt * g / L) 
        t.append((i + 1) * dt)

    N = 100
    dt = T / N
    t2 = [0]
    theta_ana = [np.pi / 3]     # initial angle
    theta_dot_ana = [0]         # initial angular velocity
    for i in range(N * T):
        theta_ana.append(theta_dot_ana[-1] * dt + theta_ana[-1])
        theta_dot_ana.append(theta_dot_ana[-1] - theta_dot_ana[-1] * dt * k / m - np.sin(theta_ana[-1]) * dt * g / L) 
        t2.append((i + 1) * dt)

    plt.plot(t, theta, label=r'Numerical solution $\theta$')
    plt.plot(t, theta_dot, label=r'Numerical solution $\dot \theta$')
    plt.plot(t2, theta_ana, label=r'Analytical solution $\theta$')
    plt.plot(t2, theta_dot_ana, label=r'Analytical solution $\dot \theta$')
    plt.xlabel('Time, $t$')
    plt.ylabel('Angular Displacement\nAngular Velocity')
    plt.legend()
    plt.show()

In [24]:
interact(plot, k = widgets.FloatSlider(min=0, max=2, value=0.8, step=0.1, description='Air resistance coefficient',readout_format='.1f',style= {'description_width': '150px'},layout = {'width': '400px'}), 
         dt = widgets.FloatSlider(min=0.01, max=1, value=0.5, step=0.04, description='Time step',style= {'description_width': '150px'},layout = {'width': '400px'}));

interactive(children=(FloatSlider(value=0.8, description='Air resistance coefficient', layout=Layout(width='40…

In [ ]:
#